<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/Mnist_CNN_ver1_0_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
# 현재 나에게 할당된 GPU 확인하기 

Fri Jun 26 18:58:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)
    

In [25]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                          transform = transforms.ToTensor(),
                         download = True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [26]:
data_loader = torch.utils.data.DataLoader(mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [31]:
class CNN(torch.nn.Module):
    def __init__(self):
      super(CNN,self).__init__() # 이부분 주의,빼먹는 경우 다수 

      self.conv1 = torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
      self.relu1 = torch.nn.ReLU()
      self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

      self.conv2 = torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
      self.relu2 = torch.nn.ReLU()
      self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

      self.fc = torch.nn.Linear(7*7*64,10,bias=True)
      torch.nn.init.kaiming_uniform(self.fc.weight)
    
    def forward(self,x):
      out = self.conv1(x)
      out = self.relu1(out)
      out = self.maxpool1(out)
      out = self.conv2(out)
      out = self.relu2(out)
      out = self.maxpool2(out)
      out = out.view(out.size(0),-1)
      out = self.fc(out)
      return out
      



In [32]:
model = CNN().to(device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  


In [33]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [34]:
total_batch = len(data_loader)
print('Learning started. It takes sometimes')
for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[Epoch {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))
print('Learning Finished!')

Learning started. It takes sometimes
[Epoch    1] cost = 0.234484807
[Epoch    2] cost = 0.0649252534
[Epoch    3] cost = 0.0474927686
[Epoch    4] cost = 0.0374915451
[Epoch    5] cost = 0.0317390226
[Epoch    6] cost = 0.0267320424
[Epoch    7] cost = 0.0227392241
[Epoch    8] cost = 0.0192376524
[Epoch    9] cost = 0.016551096
[Epoch   10] cost = 0.0136929983
[Epoch   11] cost = 0.0120437928
[Epoch   12] cost = 0.0107110124
[Epoch   13] cost = 0.00875660498
[Epoch   14] cost = 0.00867947005
[Epoch   15] cost = 0.00825472735
Learning Finished!


In [35]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy',accuracy.item())

Accuracy 0.9866999983787537


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
